# Yelp API Reviews - ETL Process

---

> *Functions, class, and methods generated by ChatGPT based on functions/workflow from prior notebook.*

**Next Steps:**

- Reintroduce the method(s) needed to get the business details beyond just the reviews.
- Polish and optimize class/workflow.\
- Add logging module for more advanced feedback and error monitoring.

---

In [2]:
import json

with open('.secrets.json') as f:
    keys = json.load(f)

client_id = keys['Client_ID']
yelp_key = keys['api_key']

In [6]:
import requests
import pandas as pd

def catch_api_errors(func):
    """Decorator to catch and report errors in API calls."""
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except requests.exceptions.RequestException as e:
            print(f"API request error in {func.__name__}: {e}")
            return None
        except Exception as e:
            print(f"General error in {func.__name__}: {e}")
            return None
    return wrapper

class YelpReviewManager:
    def __init__(self, yelp_key):
        self._yelp_key = yelp_key
        self._base_url = 'https://api.yelp.com/v3/businesses'
    
    @catch_api_errors
    def _make_api_request(self, endpoint, params=None):
        """Make a private Yelp API request."""
        headers = {'Authorization': f'Bearer {self._yelp_key}'}
        response = requests.get(f"{self._base_url}/{endpoint}", headers=headers, params=params)
        response.raise_for_status()
        return response.json()

    def fetch_businesses(self, term, location):
        """Fetch businesses matching a search term and location."""
        print(f"Fetching businesses for term '{term}' and location '{location}'...")
        businesses = []
        params = {'term': term, 'location': location, 'limit': 50}
        while True:
            response = self._make_api_request('search', params=params)
            businesses.extend(response.get('businesses', []))
            if len(businesses) >= response.get('total', 0) or not response.get('businesses'):
                break
            params['offset'] = len(businesses)
        print(f"Found {len(businesses)} businesses.")
        return [business['id'] for business in businesses]

    @catch_api_errors
    def fetch_reviews_for_business(self, business_id):
        """Fetch reviews for a single business ID."""
        print(f"Fetching reviews for business ID: {business_id}")
        reviews_response = self._make_api_request(f"{business_id}/reviews")
        return reviews_response.get('reviews', [])

    def fetch_and_compile_reviews(self, list_of_biz_ids, file_name):
        """Compile reviews for a list of business IDs."""
        print(f"Compiling reviews for {len(list_of_biz_ids)} businesses...")
        all_reviews = []
        for business_id in list_of_biz_ids:
            reviews = self.fetch_reviews_for_business(business_id)
            for review in reviews:
                review_details = pd.DataFrame([{
                    'Business ID': business_id,
                    'Reviewer Name': review['user']['name'],
                    'Review Rating': review['rating'],
                    'Review Text': review['text'],
                    'Time Created': review['time_created']
                }])
                all_reviews.append(review_details)
        compiled_reviews = pd.concat(all_reviews, ignore_index=True) if all_reviews else pd.DataFrame()
        compiled_reviews.to_csv(file_name, index=False)
        print(f"Reviews data compiled and saved to {file_name}.")

    def fetch_reviews_for_search_term(self, term, location, file_name):
        """Fetch and compile reviews for businesses matching a search term and location."""
        business_ids = self.fetch_businesses(term, location)
        if business_ids:
            self.fetch_and_compile_reviews(business_ids, file_name)
        else:
            print(f"No businesses found for term '{term}' and location '{location}'. No reviews to compile.")


In [7]:
# Initialize the YelpReviewManager with your Yelp API key
yelp_review_manager = YelpReviewManager(yelp_key=yelp_key)

# Fetch and compile reviews for hotels in Lisbon, Portugal with one method call
yelp_review_manager.fetch_reviews_for_search_term(term='hotels', location='Lisbon, Portugal', file_name='./data/lisbon_hotels_reviews.csv')

Fetching businesses for term 'hotels' and location 'Lisbon, Portugal'...
Found 268 businesses.
Compiling reviews for 268 businesses...
Fetching reviews for business ID: q-Px_TUI_zLr93xn-1-Ypw
Fetching reviews for business ID: yvbBx06xhwEyvXITlGji5g
Fetching reviews for business ID: 9wTTno6WG7cHlXDDZ00EQw
Fetching reviews for business ID: fmoWlWBqkVeyFpTtcE09RQ
Fetching reviews for business ID: DPkYK5RwCkAULtLJp7pOmA
Fetching reviews for business ID: oWpzjQrPE02mTTqb3-QuRA
Fetching reviews for business ID: UYTUME7WRO3Pz2ZiWaHCvQ
Fetching reviews for business ID: kRJ7hWBHNyItC1R3s36utQ
Fetching reviews for business ID: UNpztKHSE0A4iXnvDr2t1g
Fetching reviews for business ID: 73rCWP_Cgj5c13bPm82IsQ
Fetching reviews for business ID: kvFNVAOHH1MEP_2uJcO1Pg
Fetching reviews for business ID: jBtK8qVZ-KlH37_NhIlaQw
Fetching reviews for business ID: Yk6x3GErEUH9vXxdzmO5nQ
Fetching reviews for business ID: Dds8XUKMlB6bmW3wHfyxtQ
Fetching reviews for business ID: lIgc3taoaT4ZForepyUTqw
Fetching r